**Copyright 2020 Google LLC.**

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

In [0]:
from __future__ import division
import pandas as pd
import numpy as np
import json
import os,sys
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

## Overview

### Pre-processes UCI Adult (Census Income) dataset:

Download the Adult train and test data files can be downloaded from:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test
and save them in the `./group_agnostic_fairness/data/uci_adult` folder.

Input: 

*   ./group_agnostic_fairness/data/uci_adult/adult.data 
*   ./group_agnostic_fairness/data/uci_adult/adult.test



Outputs: train.csv, test.csv, mean_std.json, vocabulary.json, IPS_exampleweights_with_label.json, IPS_exampleweights_without_label.json

In [0]:
pd.options.display.float_format = '{:,.2f}'.format
dataset_base_dir = './group_agnostic_fairness/data/uci_adult/'

### Load original dataset

In [0]:
def convert_object_type_to_category(df):
  """Converts columns of type object to category."""
  df = pd.concat([df.select_dtypes(include=[], exclude=['object']),
                  df.select_dtypes(['object']).apply(pd.Series.astype, dtype='category')
                  ], axis=1).reindex_axis(df.columns, axis=1)
  return df

In [0]:
TRAIN_FILE = os.path.join(dataset_base_dir,'adult.data')
TEST_FILE = os.path.join(dataset_base_dir,'adult.test')

columns = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]

target_variable = "income"
target_value = ">50K"

with open(TRAIN_FILE, "r") as TRAIN_FILE:
  train_df = pd.read_csv(TRAIN_FILE,sep=',',names=columns)

with open(TEST_FILE, "r") as TEST_FILE:
  test_df = pd.read_csv(TEST_FILE,sep=',',names=columns)

In [0]:
# Convert columns of type ``object`` to ``category`` 
train_df = convert_object_type_to_category(train_df)
test_df = convert_object_type_to_category(test_df)

### Computing Invese propensity weights for each subgroup, and writes to directory.

IPS_example_weights_with_label.json: json dictionary of the format
        {subgroup_id : inverse_propensity_score,...}. Used by IPS_reweighting_model approach.

In [0]:
IPS_example_weights_without_label = {
  0: (len(train_df))/(len(train_df[(train_df.race != 'Black') & (train_df.sex != 'Female')])), # 00: White Male
  1: (len(train_df))/(len(train_df[(train_df.race != 'Black') & (train_df.sex == 'Female')])), # 01: White Female
  2: (len(train_df))/(len(train_df[(train_df.race == 'Black') & (train_df.sex != 'Female')])), # 10: Black Male
  3: (len(train_df))/(len(train_df[(train_df.race == 'Black') & (train_df.sex == 'Female')]))  # 11: Black Female
}
  
output_file_path = os.path.join(dataset_base_dir,'IPS_example_weights_without_label.json')
with open(output_file_path, mode="w") as output_file:
    output_file.write(json.dumps(IPS_example_weights_without_label))
    output_file.close()

print(IPS_example_weights_without_label)

{0: 1.6102566638642994, 1: 3.5330946180555554, 2: 20.752708731676226, 3: 20.939549839228295}


In [0]:
IPS_example_weights_with_label = {
0: (len(train_df))/(len(train_df[(train_df[target_variable] != target_value) & (train_df.race != 'Black') & (train_df.sex != 'Female')])), # 000: Negative White Male
1: (len(train_df))/(len(train_df[(train_df[target_variable] != target_value) & (train_df.race != 'Black') & (train_df.sex == 'Female')])), # 001: Negative White Female
2: (len(train_df))/(len(train_df[(train_df[target_variable] != target_value) & (train_df.race == 'Black') & (train_df.sex != 'Female')])), # 010: Negative Black Male
3: (len(train_df))/(len(train_df[(train_df[target_variable] != target_value) & (train_df.race == 'Black') & (train_df.sex == 'Female')])), # 011: Negative Black Female
4: (len(train_df))/(len(train_df[(train_df[target_variable] == target_value) & (train_df.race != 'Black') & (train_df.sex != 'Female')])), # 100: Positive White Male
5: (len(train_df))/(len(train_df[(train_df[target_variable] == target_value) & (train_df.race != 'Black') & (train_df.sex == 'Female')])), # 101: Positive White Female
6: (len(train_df))/(len(train_df[(train_df[target_variable] == target_value) & (train_df.race == 'Black') & (train_df.sex != 'Female')])), # 110: Positive Black Male
7: (len(train_df))/(len(train_df[(train_df[target_variable] == target_value) & (train_df.race == 'Black') & (train_df.sex == 'Female')])), # 111: Positive Black Female
}
  
output_file_path = os.path.join(dataset_base_dir,'IPS_example_weights_with_label.json')
with open(output_file_path, mode="w") as output_file:
    output_file.write(json.dumps(IPS_example_weights_with_label))
    output_file.close()

print(IPS_example_weights_with_label)

{0: 2.3499566974595845, 1: 4.00652147163775, 2: 25.59827044025157, 3: 22.2259385665529, 4: 5.115632364493323, 5: 29.899908172635445, 6: 109.63299663299664, 7: 361.7888888888889}


### Construct vocabulary.json, and write to directory.

vocabulary.json: json dictionary of the format {feature_name:      [feature_vocabulary]}, containing vocabulary for categorical features.

In [0]:
cat_cols = train_df.select_dtypes(include='category').columns
vocab_dict = {}
for col in cat_cols:
  vocab_dict[col] = list(set(train_df[col].cat.categories)-{"?"})
  
output_file_path = os.path.join(dataset_base_dir,'vocabulary.json')
with open(output_file_path, mode="w") as output_file:
    output_file.write(json.dumps(vocab_dict))
    output_file.close()
print(vocab_dict)

{'workclass': ['Self-emp-inc', 'Self-emp-not-inc', 'Federal-gov', 'Never-worked', 'State-gov', 'Without-pay', 'Private', 'Local-gov'], 'education': ['HS-grad', 'Doctorate', 'Masters', 'Assoc-voc', '1st-4th', '11th', '5th-6th', 'Assoc-acdm', 'Some-college', '10th', '7th-8th', '9th', 'Preschool', '12th', 'Bachelors', 'Prof-school'], 'marital-status': ['Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Never-married', 'Married-spouse-absent', 'Widowed', 'Separated'], 'occupation': ['Tech-support', 'Farming-fishing', 'Craft-repair', 'Other-service', 'Exec-managerial', 'Sales', 'Handlers-cleaners', 'Priv-house-serv', 'Prof-specialty', 'Adm-clerical', 'Armed-Forces', 'Protective-serv', 'Machine-op-inspct', 'Transport-moving'], 'relationship': ['Other-relative', 'Wife', 'Husband', 'Own-child', 'Not-in-family', 'Unmarried'], 'race': ['Amer-Indian-Eskimo', 'Other', 'White', 'Asian-Pac-Islander', 'Black'], 'sex': ['Female', 'Male'], 'native-country': ['Iran', 'Ireland', 'Japan', 'Germany', 

### Construct mean_std.json, and write to directory

mean_std.json: json dictionary of the format feature_name: [mean, std]},
containing mean and std for numerical features. 

In [0]:
temp_dict = train_df.describe().to_dict()
mean_std_dict = {}
for key, value in temp_dict.items():
  mean_std_dict[key] = [value['mean'],value['std']]

output_file_path = os.path.join(dataset_base_dir,'mean_std.json')
with open(output_file_path, mode="w") as output_file:
    output_file.write(json.dumps(mean_std_dict))
    output_file.close()
print(mean_std_dict)

{'age': [38.58164675532078, 13.640432553581341], 'fnlwgt': [189778.36651208502, 105549.97769702224], 'education-num': [10.0806793403151, 2.5727203320673877], 'capital-gain': [1077.6488437087312, 7385.292084840338], 'capital-loss': [87.303829734959, 402.9602186489998], 'hours-per-week': [40.437455852092995, 12.347428681731843]}
